**Importing dependencies**

In [5]:
import os
import re
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

**Tor-Selenium Setup**

In [6]:
torexe = os.popen(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')

firefoxBinary = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\geckodriver.exe"
proxyIP = "127.0.0.1"
proxyPort = 9150

binary = FirefoxBinary(firefoxBinary)

proxy_settings = {
    "network.proxy.type": 1,
    "network.proxy.socks": proxyIP,
    "network.proxy.socks_port": proxyPort,
    "network.proxy.socks_remote_dns": False,
    "extensions.torlauncher.start_tor": True
}

profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)

C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\4164202741.py:18: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\4164202741.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\4164202741.py:26: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\4164202741.py:26: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(executable_path=geckodriverPath,


**Scraping Lockbit**

In [7]:
# Wait till your tor browser is connected to the tor network before executing this
driver.get('http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion/')

In [8]:
SCROLL_PAUSE_TIME = 5 # as the Tor browser is slow, might need to increase this accordingly

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [9]:

base_url = "http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion/"

# Define lists to store data for both 'good' and 'bad' categories
companyNames = []
companyDescription = []
dateVictimised = []
noClicks = []

publishedStatus = []
pageURLs = []  # Store the URLs in this list

cards_good = driver.find_elements_by_xpath("//div[@class='post-block good']")

# Loop through 'good' cards
for card in cards_good:
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='views']/div[@class='updated-post-date']//span")))
    dateVictimisedElement = card.find_element_by_xpath(".//div[@class='views']/div[@class='updated-post-date']//span")
    dateVictimisedText = dateVictimisedElement.text
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element_by_xpath(".//div[@class='post-title']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='post-block-body']/div[@class='post-block-text']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element_by_xpath(".//div[@class='views']//span[@style='font-size: 12px; font-weight: bold']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

    onclick_attribute = card.get_attribute("onclick")
    match = re.search(r"return go\('([^']+)'\);", onclick_attribute)
    if match:
        relative_url = match.group(1)
        full_url = base_url + relative_url
        pageURLs.append(full_url)
        
    publishedStatus.append('Published')


cards_bad = driver.find_elements_by_xpath("//div[@class='post-block bad']")

# Loop through 'bad' cards
for card in cards_bad:
    dateVictimisedElement = card.find_element_by_xpath(".//div[@class='views']/div[@class='updated-post-date']//span")
    dateVictimisedText = dateVictimisedElement.text
    dateVictimised.append(dateVictimisedText)

    companyNameElement = card.find_element_by_xpath(".//div[@class='post-title']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)

    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='post-block-body']/div[@class='post-block-text']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)

    noClicksElement = card.find_element_by_xpath(".//div[@class='views']//span[@style='font-size: 12px; font-weight: bold']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)

    onclick_attribute = card.get_attribute("onclick")
    match = re.search(r"return go\('([^']+)'\);", onclick_attribute)
    if match:
        relative_url = match.group(1)
        full_url = base_url + relative_url
        pageURLs.append(full_url)
        
    publishedStatus.append('Not Published')

driver.quit()




C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\3972922720.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  cards_good = driver.find_elements_by_xpath("//div[@class='post-block good']")
C:\Users\Melisse\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\3972922720.py:44: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  cards_bad = driver.find_elements_by_xpath("//div[@class='post-block bad']")


In [10]:

# Create a DataFrame
# df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, 
#                    "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus,
#                   "Actual Date": actualDate})
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, 
                   "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus,
                  "pageurl":pageURLs})

# Convert the 'Date Victimised' column to datetime
df['Date Victimised'] = pd.to_datetime(df['Date Victimised'], format='Updated: %d %b, %Y, %H:%M UTC')
# df['Actual Date'] = pd.to_datetime(df['Actual Date'], format='%d %b, %Y %H:%M UTC')


# Filter data where the year is less than 2023
date_mask = df['Date Victimised'].dt.year < 2023
# date_mask = df['Actual Date'].dt.year < 2023
df = df[~date_mask]


# Reset the index
df = df.reset_index(drop=True)

# Print the summary of the DataFrame
print("Summary of the DataFrame:")
print(df.info())

# Print the number of rows and columns
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Check for missing (empty) cells
missing_cells = df.isnull().sum().sum()
if missing_cells > 0:
    print(f"Number of missing cells: {missing_cells}")
else:
    print("No missing cells in the DataFrame")

# Ensure all data is displayed
pd.set_option('display.max_rows', None)
print(df)

Summary of the DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Company              997 non-null    object        
 1   Company Description  997 non-null    object        
 2   Date Victimised      997 non-null    datetime64[ns]
 3   Number of Clicks     997 non-null    object        
 4   Status               997 non-null    object        
 5   pageurl              997 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 46.9+ KB
None
Number of rows: 997
Number of columns: 6
No missing cells in the DataFrame
                                  Company  \
0                               sirva.com   
1                                cdwg.com   
2                                cdwg.com   
3                      foremostgroups.com   
4                              fdf.org.uk   
5   

**Scraping for date**

In [15]:
import csv

# initialise list to store links
url_list = []

#Read csv file
with open('Lockbit_Data1.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)

    next(csvreader)

    for row in csvreader:
        if len(row) >=8:
            pageurl = row[7]
            url_list.append(pageurl)

print(url_list)
            

['http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/RbVJA9aK70XvgWYu651b4b2a183ec', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/8PRrYsjGQLIHjp5u6526ec1587274', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/bEPilXlXUFyBrRwN6526eb74b293c', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/PQIrggtu9XcMkvKz65246b0f230d3', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/pJt2ilA0kzfg3nIM6515e0c4e8207', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/aQ965JJyVTOCFkO16519e8a256bb8', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/zwi3XWuSI7P03Pdr6519e7b6eb6f5', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/BiucT5vh19VpZs0a64fef42b33dbd', 'http://lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/rTSg7aUW0Qq9E3Lv64f1a27a7dd69', 'http://lockbitapt

In [32]:
from selenium.webdriver.chrome.options import Options



# Specify paths
firefoxBinary = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\geckodriver.exe"

# Set up the Firefox binary and profile
binary = FirefoxBinary(firefoxBinary)
profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference('network.proxy.socks_remote_dns', True)
profile.update_preferences()

# Create a single driver instance
driver = webdriver.Firefox(executable_path=geckodriverPath, firefox_binary=binary, firefox_profile=profile)

# Set up headless mode
options = webdriver.FirefoxOptions()
options.headless = True
time.sleep(10)
actualDate = []

# Important to do in batches, if not, the browser may crash
for counter in range(800, 997):  # Modify the range as needed
    u = url_list[counter]
    try:
        print(counter)
        driver.get(u)
        if counter == 0:
            timeout = 50 
        else:
            timeout = 15
        loaded = WebDriverWait(driver, timeout).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='uploaded-date-utc']")))
        AD = driver.find_element_by_class_name("uploaded-date-utc")
        actualDate.append(AD.text)
        time.sleep(10)

        # if counter % 5 == 0:
        #     # Every 5 URLs, reset the driver to prevent slowdown
        #     print("Quitting... Starting a new one")

        #     # Save data to CSV
        #     df_actual = pd.DataFrame()
        #     df_actual['ActualDate'] = actualDate
        #     df_actual.to_csv('ActualDate.csv')

        #     # Reinitialize the driver
        #     driver = webdriver.Firefox(executable_path=geckodriverPath, firefox_binary=binary, firefox_profile=profile)

    except Exception as e:
        print(e)
        actualDate.append("Not found")

# Close the driver and save the remaining data
driver.quit()
# df_actual = pd.DataFrame()
# df_actual['ActualDate'] = actualDate
# df_actual.to_csv('ActualDate.csv')


C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\2505244429.py:11: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\2505244429.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath, firefox_binary=binary, firefox_profile=profile)
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\2505244429.py:19: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath, firefox_binary=binary, firefox_profile=profile)
C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\2505244429.py:19: DeprecationWarning: firefox_profile has been deprecated, please pass in an Op

800


C:\Users\Melisse\AppData\Local\Temp\ipykernel_14480\2505244429.py:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  AD = driver.find_element_by_class_name("uploaded-date-utc")


801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:674:5
bail@chrome://remote/content/marionette/sync.sys.mjs:213:19

819
Message: Reached error page: about:neterror?e=connectionFailure&u=http%3A//lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion//post/bFfZ9yTRtDhTlYzw632251b34d1ce&c=UTF-8&d=The%20connection%20was%20refused%20when%20attempting%20to%20contact%20lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:736:5
checkReadyState@chrome://remote/conte

In [33]:
df_actual = pd.DataFrame()
df_actual['ActualDate'] = actualDate
print(df_actual)
df_actual.to_csv("AD13(800-997).csv")

                 ActualDate
0    21 SEP, 2022 15:26 UTC
1    21 SEP, 2022 15:24 UTC
2    21 SEP, 2022 15:23 UTC
3    21 SEP, 2022 15:07 UTC
4    20 SEP, 2022 08:08 UTC
5    19 SEP, 2022 20:46 UTC
6    19 SEP, 2022 20:43 UTC
7    19 SEP, 2022 20:34 UTC
8    19 SEP, 2022 20:13 UTC
9    19 SEP, 2022 18:31 UTC
10   19 SEP, 2022 09:25 UTC
11   17 SEP, 2022 02:39 UTC
12   16 SEP, 2022 03:25 UTC
13   16 SEP, 2022 03:23 UTC
14   16 SEP, 2022 03:17 UTC
15   16 SEP, 2022 03:16 UTC
16   16 SEP, 2022 00:03 UTC
17   15 SEP, 2022 15:05 UTC
18                Not found
19                Not found
20   14 SEP, 2022 22:08 UTC
21   14 SEP, 2022 22:06 UTC
22   14 SEP, 2022 20:38 UTC
23   14 SEP, 2022 20:15 UTC
24   14 SEP, 2022 20:09 UTC
25   14 SEP, 2022 19:32 UTC
26   14 SEP, 2022 19:13 UTC
27   14 SEP, 2022 17:43 UTC
28   14 SEP, 2022 02:23 UTC
29   14 SEP, 2022 02:22 UTC
30   14 SEP, 2022 02:20 UTC
31   14 SEP, 2022 02:19 UTC
32   14 SEP, 2022 01:41 UTC
33   14 SEP, 2022 01:40 UTC
34   14 SEP, 2022 01

In [108]:
from selenium.webdriver.chrome.options import Options

firefoxBinary = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\geckodriver.exe"


binary = FirefoxBinary(firefoxBinary)

# proxy_settings = {
#     "network.proxy.type": 1,
#     "network.proxy.socks": proxyIP,
#     "network.proxy.socks_port": proxyPort,
#     "network.proxy.socks_remote_dns": False,
#     "extensions.torlauncher.start_tor": True
# }

profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
# profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

options = Options()
options.headless = True

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)
time.sleep(10)


actualDate = []
#important to do in batches, if not kali will crash 
for counter in range(0, 5): #till 170  max length:180 (0 to 179)
    u = pageURLs[counter]
    try:
        print(counter)
        driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)
        website = driver.get(u)
        loaded = WebDriverWait(driver, 15).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='uploaded-date-utc']"))) 
        AD = driver.find_element_by_class_name("uploaded-date-utc")
        actualDate.append(AD.text)
        time.sleep(10)
        if counter % 5 == 0:
            #every 5 urls reset the driver, to prevent slow down 
            print("Quitting... Starting a new one") 
            driver.quit()
            
            df_actual = pd.DataFrame()
            df_actual['ActualDate'] = actualDate
            df_actual.to_csv('ActualDate.csv')
            
            time.sleep(10)
            firefoxBinary = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\firefox.exe"
            geckodriverPath = r"C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\geckodriver.exe"

            binary = FirefoxBinary(firefoxBinary)

            profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
            profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
            profile.set_preference('network.proxy.type', 1)
            profile.set_preference('network.proxy.socks', '127.0.0.1')
            profile.set_preference('network.proxy.socks_port', 9150)
            profile.set_preference("network.proxy.socks_remote_dns", True)
            profile.update_preferences()
            options = Options()
            options.headless = True
    except Exception as e:
        print(e)
        actualDate.append("Not found")



driver.quit()

C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:17: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = FirefoxProfile(r'C:\Users\Melisse\OneDrive - Nanyang Technological University\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:28: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:28: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(executable_path=geckodriverPath,


0


C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:40: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:40: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=geckodriverPath,
C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:40: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(executable_path=geckodriverPath,


Message: 

1
Message: 

2


C:\Users\Melisse\AppData\Local\Temp\ipykernel_12964\1046420190.py:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  AD = driver.find_element_by_class_name("uploaded-date-utc")


3
4


In [109]:
df_actual = pd.DataFrame()
df_actual['ActualDate'] = actualDate
print(df_actual)
df_actual.to_csv("AD2.csv")


               ActualDate
0               Not found
1               Not found
2  11 OCT, 2023 18:38 UTC
3  09 OCT, 2023 21:05 UTC
4  28 SEP, 2023 20:24 UTC


In [91]:
df4.to_csv("Lockbit_Data2.csv")

NameError: name 'df4' is not defined

In [87]:

# Create a DataFrame
# df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, 
#                    "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus,
#                   "Actual Date": actualDate})
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, 
                   "Date Victimised": dateVictimised, "Number of Clicks": noClicks, "Status": publishedStatus,
                  "pageurl":pageURLs})

# Convert the 'Date Victimised' column to datetime
df['Date Victimised'] = pd.to_datetime(df['Date Victimised'], format='Updated: %d %b, %Y, %H:%M UTC')
# df['Actual Date'] = pd.to_datetime(df['Actual Date'], format='%d %b, %Y %H:%M UTC')


# Filter data where the year is less than 2023
date_mask = df['Date Victimised'].dt.year < 2023
# date_mask = df['Actual Date'].dt.year < 2023
df = df[~date_mask]


# Reset the index
df = df.reset_index(drop=True)

# Print the summary of the DataFrame
print("Summary of the DataFrame:")
print(df.info())

# Print the number of rows and columns
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Check for missing (empty) cells
missing_cells = df.isnull().sum().sum()
if missing_cells > 0:
    print(f"Number of missing cells: {missing_cells}")
else:
    print("No missing cells in the DataFrame")

# Ensure all data is displayed
pd.set_option('display.max_rows', None)
print(df)

Summary of the DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Company              998 non-null    object        
 1   Company Description  998 non-null    object        
 2   Date Victimised      998 non-null    datetime64[ns]
 3   Number of Clicks     998 non-null    object        
 4   Status               998 non-null    object        
 5   pageurl              998 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 46.9+ KB
None
Number of rows: 998
Number of columns: 6
No missing cells in the DataFrame
                                  Company                                Company Description     Date Victimised Number of Clicks         Status                                            pageurl
0                               sirva.com  For media: contact us please. We wa

In [88]:
df.to_csv("Lockbit_Data1.csv")